# <a name="0"><font color='Blue'>**Rossmann Store Sales**</font></a>
## **A Time Series problem.**
### Dataset used is from a Kaggle competition [this](https://www.kaggle.com/competitions/rossmann-store-sales) link.

---

## **Abstract:**

**Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.**


**The notebook explores different methods to forecast sales for Rossmann stores. The models evaluated include machine learning (ML) models and time series models, as well as simple models for comparison.**

---

## **Table of Contents of the notebook:**

1. <a href="#1">**Setup**</a>
2. <a href="#2">**Reading the dataset**</a>
3. <a href="#3">**Data Preperation**</a>
4. <a href="#4">**Train Test Split**</a>
5. <a href="#5">**Gaining Insights (EDA)**</a>
6. <a href="#6">**Time Series Models**</a>
7. <a href="#7">**Machine Leaning Model**</a>
8. <a href="#8">**Choosing Best Model**</a>

---

# 1. <a name="1">**Setup**</a>
(<a href="#0">Go to top</a>)

**Installing and Importing used libraries.**

In [ ]:
# Install pmdarima
!pip install pmdarima

In [ ]:
# Install statsmodels
!pip install git+https://github.com/statsmodels/statsmodels.git@74836e9cf2198ac7a930146405da7239c0823a9b#egg=statsmodels

In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import MSTL
from statsmodels.tsa.stattools import kpss,adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from lightgbm import LGBMRegressor
from prophet import Prophet
logging.getLogger("cmdstanpy").disabled = True 
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
from itertools import product

---

# 2. <a name="2">**Reading the dataset**</a>
(<a href="#0">Go to top</a>)

In [ ]:
df = pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv', index_col = "Date", low_memory=False, parse_dates=['Date'])

In [ ]:
df.head()

In [ ]:
df.info()

---

# 3. <a name="3">**Data preperation**</a>
(<a href="#0">Go to top</a>)

## A- Choose The Store We will work on (Store 1)

In [ ]:
df = df[df["Store"] == 1]

## B- Check for nulls

In [ ]:
df.isnull().sum()

## C- Make freq as daily

In [ ]:
df = df.asfreq('D')

In [ ]:
df.isnull().sum()

## D- Remove Customers column as it won't be available in production

In [ ]:
df.drop(["Customers"], axis=1,inplace=True)

## E- Sort by date

In [ ]:
df.sort_values(by="Date", inplace=True)

---

# 4. <a name="4">**Train Test Split**</a>
(<a href="#0">Go to top</a>)

### <font color='green'>**Split data into train and test 80:20**</font>

In [ ]:
train_size = int(len(df) * 0.8)
train_data = df[:train_size].copy(deep=True)
test_data = df[train_size:].copy(deep=True)

---

# 5. <a name="5">**Gaining Insights (EDA)**</a>
(<a href="#0">Go to top</a>)

## A- Plot Sales (All train data) 

In [ ]:
plt.figure(figsize=(10, 6))
train_data['Sales'].plot()
plt.xlabel('Day')
plt.ylabel('Sales')
plt.title('Sales over Time (Whole data)')
plt.show()

## B- Plot Sales (One Month) 

In [ ]:
plt.figure(figsize=(10, 6))
train_data['Sales'][:30].plot()
plt.xlabel('Day')
plt.ylabel('Sales')
plt.title('Sales over Time (One Month)')
plt.show()

### <font color='green'>**It seems that Friday is the off day of the stores**</font>

## C- Plot Sales (One Year) 

In [ ]:
plt.figure(figsize=(10, 6))
train_data['Sales'][:365].plot()
plt.xlabel('Day')
plt.ylabel('Sales')
plt.title('Sales over Time (One Year)')
plt.show()

### <font color='green'>**We see a peak in sales in Dec**</font>

## D- Extract additional columns 

In [ ]:
train_data['year'] = train_data.index.year
train_data['month'] = train_data.index.month
train_data['day'] = train_data.index.day

# Convert the columns to integers
train_data['year'] = train_data['year'].astype(int)
train_data['month'] = train_data['month'].astype(int)
train_data['day'] = train_data['day'].astype(int)

In [ ]:
train_data.info()

## E- Plot sales by different dates values

In [ ]:
# Create a grid of bar charts
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
sns.barplot(x='year',y="Sales", data=train_data, ax=axes[0, 0])
sns.barplot(x='month',y="Sales", data=train_data, ax=axes[0, 1])
sns.barplot(x='day',y="Sales", data=train_data, ax=axes[1, 0])
sns.barplot(x='DayOfWeek',y="Sales", data=train_data, ax=axes[1, 1])

# Set the titles for each chart
axes[0, 0].set_title('Sales by Year')
axes[0, 1].set_title('Sales by Month')
axes[1, 0].set_title('Sales by Day')
axes[1, 1].set_title('Sales by Weekday')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the grid of bar charts
plt.show()

### <font color='green'>**The Insighst we gained:**<Br><Br>1. Sales decrease over the years<Br><Br>2. The peak of sales is on Nov and Dec<Br><Br>3. Sales increase in the opening and closing day of the week</font>

## F- Define Monthly seasonal plot

In [ ]:
def monthly_quarter_line_plot(df,col_x,col_y,hue_col,title="Monthly"):
    plt.figure(figsize=(10,8))
    sns.lineplot(data=df, 
                 x=col_x, 
                 y=col_y, 
                 hue=hue_col, 
                 legend='full',palette="tab10")

    # add title
    plt.title(title+' Seasonal plot')

    # move the legend outside of the main figure
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2)

## G- plot Monthly seasonal

In [ ]:
monthly_quarter_line_plot(train_data,"month","Sales","year",title="Monthly")

## H- Plot Sales in different conditions

In [ ]:
# Create a grid of bar charts
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
sns.barplot(x='Open',y="Sales", data=train_data, ax=axes[0, 0])
sns.barplot(x='Promo',y="Sales", data=train_data, ax=axes[0, 1])
sns.barplot(x='StateHoliday',y="Sales", data=train_data, ax=axes[1, 0])
sns.barplot(x='SchoolHoliday',y="Sales", data=train_data, ax=axes[1, 1])

# Set the titles for each chart
axes[0, 0].set_title('Sales in Open')
axes[0, 1].set_title('Sales in Promo')
axes[1, 0].set_title('Sales in StateHoliday')
axes[1, 1].set_title('Sales in SchoolHoliday')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the grid of bar charts
plt.show()

### <font color='green'>**Nothing unusual, no sales when store is closed, State holidays don't add new information, sales increase in promo, and Shool holidays don't make that difference**</font>

## I- Define histogram function

In [ ]:
def histograms(df,col_name,bins_number=100,diff=False, xmin=-1000, xmax=1000):
    if diff:
        plt.figure(figsize=(8,6))
        plt.hist(df[col_name].diff(),bins=bins_number)
        plt.xlim(xmin,xmax)
        plt.show()
    else:
        plt.figure(figsize=(8,6))
        plt.hist(df[col_name],bins=bins_number)
        plt.show()

## J- Distribution of sales values differences (When the store is not closed)

In [ ]:
histograms(train_data[train_data["Sales"] != 0],"Sales",bins_number=200,diff=True)

## K- Distribution of sales values (When the store is not closed)

In [ ]:
histograms(train_data[train_data["Sales"] != 0],"Sales",bins_number=200,diff=False)

---

# 6. <a name="6">**Time Series Models**</a>
(<a href="#0">Go to top</a>)

![](https://forums.fast.ai/uploads/default/original/2X/3/3c449523e3973a978d480854fd17af0299b5c9b7.png)

### Eval Metric (RMSPE):

In [ ]:
def rmspe(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred)) / np.mean(y_true)

### Plot train vs test

In [ ]:
fig ,axes =plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes.plot(train_data["Sales"], label='Train',color='blue')
axes.plot(test_data["Sales"], label='Test',color='orange')
plt.show()

## **A- Simple Forecasting Models**

### Models Functions

In [ ]:
def average_method(train_data,test_data):
    ## Average method 
    average_prediction = [np.mean(train_data)]*len(test_data)
    average_pred = pd.DataFrame(average_prediction)
    average_pred.index = test_data.index
    return average_pred.squeeze()

def naive_method(train_data,test_data):
    ##Naive method
    naiive_prediction = [train_data.iloc[-1]]*len(test_data)
    naiive_pred = pd.DataFrame(naiive_prediction)
    naiive_pred.index = test_data.index
    return naiive_pred.squeeze()

def seasonal_naive(train_data,test_data):
    ##SEasonal_NAive
    dates = (test_data.index - np.timedelta64(1, 'Y')).values.astype('datetime64[D]')
    dates = dates + np.timedelta64(2,'D')
    seasonal_naive_prediction = train_data[train_data.index.isin(dates)].values # seasonal naive prediction
    seasonal_naive = pd.DataFrame(seasonal_naive_prediction).set_index(test_data.index)
    return seasonal_naive.squeeze()

def drift_method(train_data,test_data):
    # Get the slope
    y_t = train_data[len(train_data)-1]
    m = (y_t - train_data[1]) / len(train_data)
    h = np.linspace(0,len(test_data)-1, len(test_data))
    drift_prediction = y_t + m * h
    drift_pred = pd.DataFrame(drift_prediction).set_index(test_data.index)
    return drift_pred.squeeze()

### Get predictions

In [ ]:
average_pred = average_method(train_data["Sales"],test_data["Sales"])
naiive_pred = naive_method(train_data["Sales"],test_data["Sales"])
seasonal_naive_pred = seasonal_naive(train_data["Sales"],test_data["Sales"])
drift_pred = drift_method(train_data["Sales"],test_data["Sales"])

### Make Value of prediction = 0 when The Store is not open

In [ ]:
for pred in [average_pred,naiive_pred,seasonal_naive_pred,drift_pred]:
    pred[test_data["Open"] == 0] = 0

### Plot predicions

In [ ]:
fig ,axes =plt.subplots(2,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes[0].plot(train_data["Sales"], label='Train',color='blue')
axes[0].plot(test_data["Sales"], label='Test',color='orange')

axes[0].plot(average_pred,label="Average method",color='red')
axes[0].plot(naiive_pred,label="Naive method",color='purple')
axes[0].plot(seasonal_naive_pred, label='Seasonal_Naive',color='green')
axes[0].plot(drift_pred,label='Drift',color='orchid')
axes[0].legend(loc='best')

axes[1].plot(test_data["Sales"], label='Test',color='orange')
axes[1].plot(average_pred,label="Average method",color='red')
axes[1].plot(naiive_pred,label="Naive method",color='purple')
axes[1].plot(seasonal_naive_pred, label='Seasonal_Naive',color='green')
axes[1].plot(drift_pred,label='Drift',color='orchid')
axes[1].legend(loc='best')
plt.show()

### Evaluate predictions

In [ ]:
print(f"""
RMSPE for average method:{rmspe( average_pred, test_data[["Sales"]])}
RMSPE for Naive method:{rmspe( naiive_pred, test_data[["Sales"]])}
RMSPE for Seasonal_Naive method:{rmspe( seasonal_naive_pred, test_data[["Sales"]])}
RMSPE for Drift method:{rmspe( drift_pred, test_data[["Sales"]])}""")

### <font color='green'>**Seasonal naive method is doing a very good job actually!**</font>

## **B- MSTL Model**

In [ ]:
mstl = MSTL(train_data["Sales"], periods=(7, 30, 365), stl_kwargs={"seasonal_deg": 0})
res = mstl.fit() 

### <font color='green'>**Multiple seasonal periods are allowed, so we will choose weekly, monthly and anually**</font>

In [ ]:
# Start with the plot from the results object `res`
plt.rc("figure", figsize=(16, 20))
plt.rc("font", size=13)
fig = res.plot()

# Make plot pretty
axs = fig.get_axes()

axs[0].set_ylabel("Sales")
axs[0].set_title("Decomposition sales")

plt.tight_layout()

<div class="alert-info">
<font color='black'>Fitting functions on seasonal and trend decomposition of time series (STL) can be challenging, Despite the challenges, STL decomposition remains a useful technique for analyzing and modeling time series data, especially when dealing with seasonal and trend components</font>
</div>

## **C- Arima models**

### Define kpss_test function

In [ ]:
def kpss_test(data,threshold=0.05,regression="c"):
    result = kpss(data,regression="c")
    #print(result)
    # Print test results
    print('KPSS Statistic:', result[0])
    print('p-value:', result[1])
    print('Lags Used:', result[2])
    print('Critical Values:')
    for key, value in result[3].items():
        print('\t{}: {}'.format(key, value))
    if result[1]<=threshold:
        print("The data is not stationary")
    else:
        print("The data is stationary")

### Define adf_test function

In [ ]:
def adf_test(data,threshold=0.05):
    # Perform ADF test
    result = adfuller(data,autolag="AIC")
    
    # Print test results
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))
    
    if result[1] <= threshold:
        print("Conclusion:====>") 
        print("Reject the null hypothesis") 
        print("Data is stationary")
    else:
        print("Conclusion:====>")
        print("Fail to reject the null hypothesis") 
        print("Data is non-stationary")

### Define autocorrelatin_graphs function

In [ ]:
def autocorrelatin_graphs(value,n_lags,title_text):
    fig, axes = plt.subplots(3,1,dpi=80)
    fig.set_figheight(12)
    fig.set_figwidth(16)
    value_plot = axes[0].plot(value)
    plt.title(f'{title_text}')
    acf_plot = plot_acf(value, lags=n_lags, title=f'Autocorrelation in {title_text}',ax=axes[1])
    plt.xlabel('Lags')
    pacf_plot = plot_pacf(value, lags=n_lags, title=f'Partial Autocorrelation in {title_text}',ax=axes[2])
    plt.xlabel('Lags')
    plt.tight_layout()
    plt.show()

### Plot Rolling mean and STD

In [ ]:
fig ,axes =plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(16)

#Determing rolling statistics
rolmean = train_data[["Sales"]].rolling( window=30).mean()
rolstd = train_data[["Sales"]].rolling(window=30).std()

#Plot rolling statistics:
orig = axes.plot(train_data[["Sales"]], color='blue',label='Original')
mean = axes.plot(rolmean, color='red', label='Rolling Mean')
std = axes.plot(rolstd, color='black', label = 'Rolling Std')
axes.legend(loc='best')
axes.set_title('Rolling Mean & Standard Deviation')

### Check stationarity:

In [ ]:
adf_test(train_data[["Sales"]])

In [ ]:
kpss_test(train_data[["Sales"]])

### <font color='green'>**In the first look, data seems to be non stationary. Despite that, the tests confirm that the data is stationary. The STD is high but is almost steady across all the winows**</font>

### Autocorrelatin Graphs

In [ ]:
autocorrelatin_graphs(train_data[["Sales"]], n_lags=40, title_text="Autocorrelatin Graphs of sales")

### <font color='green'>**The ACF and PACF show a sine wave-like correlation pattern with high positive correlations at lag 7, 14, 21, and so on, it suggests the presence of a seasonal component in the data.**</font>

### Arima

In [ ]:
# fit model
ARIMA_model = ARIMA(train_data[["Sales"]], order=(1, 1, 1))
ARIMA_model_fit = ARIMA_model.fit()

In [ ]:
ARIMA_model_fit.summary()

In [ ]:
start_index = test_data.index.min()
end_index = test_data.index.max()

#Predictions
arima_preds = ARIMA_model_fit.predict(start=start_index, end=end_index)

### Make Value of prediction = 0 when The Store is not open

In [ ]:
arima_preds[test_data["Open"] == 0] = 0

### Evaluate performance

In [ ]:
print(f'RMSPE for Arima on test:{rmspe(arima_preds,test_data[["Sales"]])}')

In [ ]:
fig ,axes =plt.subplots(2,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes[0].plot(train_data["Sales"], label='Train',color='blue')
axes[0].plot(test_data["Sales"], label='Test',color='orange')

axes[0].plot(arima_preds,label="Arima",color='red')
axes[0].legend(loc='best')

axes[1].plot(test_data["Sales"], label='Test',color='orange')
axes[1].plot(arima_preds,label="Arima",color='red')
axes[1].legend(loc='best')
plt.show()

### SARIMAX

#### <font color='green'>**Considering the characteristics, an appropriate model to capture the seasonality and the sine wave-like patterns would be the SARIMA (Seasonal Autoregressive Integrated Moving Average) model. SARIMA is an extension of the ARIMA model that incorporates seasonal components.**</font>

In [ ]:
# fit model
SARIMAX_model = SARIMAX(train_data[["Sales"]], order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
SARIMAX_model_fit = SARIMAX_model.fit()

In [ ]:
SARIMAX_model_fit.summary()

In [ ]:
start_index = test_data.index.min()
end_index = test_data.index.max()

#Predictions
SARIMAX_preds = SARIMAX_model_fit.predict(start=start_index, end=end_index)

### Make Value of prediction = 0 when The Store is not open

In [ ]:
SARIMAX_preds[test_data["Open"] == 0] = 0

In [ ]:
print(f'RMSPE for SARIMAX on test:{rmspe(SARIMAX_preds,test_data[["Sales"]])}')

In [ ]:
fig ,axes =plt.subplots(2,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes[0].plot(train_data["Sales"], label='Train',color='blue')
axes[0].plot(test_data["Sales"], label='Test',color='orange')

axes[0].plot(SARIMAX_preds,label="SARIMAX",color='red')
axes[0].legend(loc='best')

axes[1].plot(test_data["Sales"], label='Test',color='orange')
axes[1].plot(SARIMAX_preds,label="SARIMAX",color='red')
axes[1].legend(loc='best')
plt.show()

### Auto Arima

In [ ]:
stepwise_model = auto_arima(train_data[["Sales"]],start_p=1, start_q=1,d=1, max_p=3, max_q=3,
                            seasonal=True, start_P =1, start_Q=1, D=1, max_Q=3, max_P=3, max_order=12, m=7,
                            trace=True,stationary=True,error_action='ignore',
                            suppress_warnings=True, stepwise=True)
stepwise_model.summary()

In [ ]:
length = len(test_data)

#Predictions
auto_arima_forecast,conf_int = stepwise_model.predict(n_periods=length,return_conf_int=True)

### Make Value of prediction = 0 when The Store is not open

In [ ]:
auto_arima_forecast[test_data["Open"] == 0] = 0

In [ ]:
print(f'RMSPE for Auto Arima on test:{rmspe(auto_arima_forecast,test_data[["Sales"]])}')

In [ ]:
fig ,axes =plt.subplots(2,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes[0].plot(train_data["Sales"], label='Train',color='blue')
axes[0].plot(test_data["Sales"], label='Test',color='orange')

axes[0].plot(auto_arima_forecast,label="Auto Arima",color='red')
axes[0].legend(loc='best')

axes[1].plot(test_data["Sales"], label='Test',color='orange')
axes[1].plot(auto_arima_forecast,label="Auto Arima",color='red')
axes[1].legend(loc='best')
plt.show()

## **D- Prophet**

### Define preparing data function

In [ ]:
def prepare_df_prophet(df, Type="train"):
    new_df = df.copy(deep=True)
    new_df.drop(["Store","StateHoliday"], inplace=True, axis=1)
    new_df['Day'] = new_df.index.day.astype(int)
    new_df['Month'] = new_df.index.month.astype(int)
    new_df['Year'] = new_df.index.year.astype(int)
    new_df['DayOfYear'] = new_df.index.dayofyear.astype(int)
    new_df['WeekOfYear'] = new_df.index.isocalendar().week.astype(int)
    new_df = new_df.reset_index()
    new_df.rename({"Sales": "y", "Date" : "ds"},axis=1, inplace = True)
    if Type == "test":
        new_df.drop("y", axis=1, inplace=True)
    return new_df

In [ ]:
train_prophet = prepare_df_prophet(train_data)

In [ ]:
train_prophet.columns

### Create model instance and adding all features as regressors

In [ ]:
Prophet_model = Prophet()

regressors = ['DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'Day',
              'Month', 'Year', 'DayOfYear', 'WeekOfYear']

for regressor in regressors:
    Prophet_model.add_regressor(regressor)

### Model fitting

In [ ]:
Prophet_model.fit(train_prophet)

### Prepare test dataset

In [ ]:
test_prophet = prepare_df_prophet(test_data, Type="test")

In [ ]:
forecast = Prophet_model.predict(test_prophet)

In [ ]:
prophet_preds = forecast.copy(deep=True)
prophet_preds = prophet_preds.set_index("ds")["yhat"].clip(lower=0)

### Make Value of prediction = 0 when The Store is not open

In [ ]:
prophet_preds[test_data["Open"] == 0] = 0

### Evaluate performance

In [ ]:
print(f'RMSPE for Prophet on Test:{rmspe(prophet_preds,test_data["Sales"])}')

### Plot predictions

In [ ]:
fig ,axes =plt.subplots(2,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes[0].plot(train_data["Sales"], label='Train',color='blue')
axes[0].plot(test_data["Sales"], label='Test',color='orange')

axes[0].plot(prophet_preds,label="Prophet",color='red')
axes[0].legend(loc='best')

axes[1].plot(test_data["Sales"], label='Test',color='orange')
axes[1].plot(prophet_preds,label="Prophet",color='red')
axes[1].legend(loc='best')
plt.show()

### Fine Tuning

In [ ]:
regressors = ['DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'Day',
              'Month', 'Year', 'DayOfYear', 'WeekOfYear']

train_prophet = prepare_df_prophet(train_data)

test_prophet = prepare_df_prophet(test_data, Type="test")

# Define the parameter grid
param_grid = {
    'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0],
    'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
    'changepoint_range': [0.8, 0.9, 1.0],
}

best_model = None
best_param = None
best_rmspe = float('inf')

# Iterate over all combinations of parameters
for params in tqdm(ParameterGrid(param_grid)):
    # Create a new Prophet model with the specified parameters
    model = Prophet(**params)

    # Add regressors to the model
    for regressor in regressors:
        model.add_regressor(regressor)

    # Fit the model to the training data
    model.fit(train_prophet)

    # Make predictions on the test data
    forecast = model.predict(test_prophet)
    prophet_preds = forecast.set_index("ds")["yhat"].clip(lower=0)
    prophet_preds[test_data["Open"] == 0] = 0

    # Calculate RMSPE
    rmspe_score = rmspe(prophet_preds, test_data["Sales"])

    # Check if the current model is the best so far
    if rmspe_score < best_rmspe:
        best_rmspe = rmspe_score
        print(f"  Found new best RMSPE: {best_rmspe}")
        best_model = model
        best_param = params

# Print the RMSPE of the best model
print(f"Best RMSPE: {best_rmspe}")
print(f"Best Params: {best_param}")

In [ ]:
forecast = best_model.predict(test_prophet)

### Evaluate performance of best model

In [ ]:
prophet_preds = forecast.copy(deep=True)
prophet_preds = prophet_preds.set_index("ds")["yhat"].clip(lower=0)
print(f'RMSPE for Prophet on Test:{rmspe(prophet_preds,test_data["Sales"])}')

### Plot predictions of best model

In [ ]:
fig ,axes =plt.subplots(2,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes[0].plot(train_data["Sales"], label='Train',color='blue')
axes[0].plot(test_data["Sales"], label='Test',color='orange')

axes[0].plot(prophet_preds,label="Prophet",color='red')
axes[0].legend(loc='best')

axes[1].plot(test_data["Sales"], label='Test',color='orange')
axes[1].plot(prophet_preds,label="Prophet",color='red')
axes[1].legend(loc='best')
plt.show()

---

# 7. <a name="7">**Machine Learning model**</a>
(<a href="#0">Go to top</a>)

In [ ]:
train_size = int(len(df) * 0.8)
train_data = df[:train_size].copy(deep=True)
test_data = df[train_size:].copy(deep=True)

### Prepare df function

In [ ]:
def prepare_df_ml(df):
    new_df = df.copy(deep=True)
    new_df.drop(["Store","StateHoliday"], inplace=True, axis=1)
    new_df['Day'] = new_df.index.day.astype(int)
    new_df['Month'] = new_df.index.month.astype(int)
    new_df['Year'] = new_df.index.year.astype(int)
    new_df['DayOfYear'] = new_df.index.dayofyear.astype(int)
    new_df['WeekOfYear'] = new_df.index.isocalendar().week.astype(int)
    df_prep_X = new_df.drop("Sales", axis=1)
    df_prep_Y = new_df["Sales"]
    return df_prep_X, df_prep_Y

### Prepare training data

In [ ]:
train_prep_X, train_prep_Y = prepare_df_ml(train_data)

In [ ]:
train_prep_X.head()

In [ ]:
train_prep_Y

### Model fitting

In [ ]:
lgbm_model = LGBMRegressor(random_state=42)

In [ ]:
train_prep_X.info()

In [ ]:
lgbm_model.fit(train_prep_X, train_prep_Y)

### Performance on train

In [ ]:
lgbm_train_preds = lgbm_model.predict(train_prep_X)
lgbm_train_preds = pd.Series(lgbm_train_preds, index= train_prep_X.index)
lgbm_train_preds = lgbm_train_preds.clip(lower=0)
lgbm_train_preds[train_prep_X["Open"] == 0] = 0
print(f'RMSPE for LGBM on Train:{rmspe(lgbm_train_preds,train_prep_Y)}')

### Prepare testing data

In [ ]:
test_prep_X, test_prep_Y = prepare_df_ml(test_data)

In [ ]:
test_prep_X

In [ ]:
lgbm_preds = lgbm_model.predict(test_prep_X)

### Performance on test

In [ ]:
lgbm_test_preds = lgbm_model.predict(test_prep_X)
lgbm_test_preds = pd.Series(lgbm_test_preds, index= test_prep_X.index)
lgbm_test_preds = lgbm_test_preds.clip(lower=0)
lgbm_test_preds[test_prep_X["Open"] == 0] = 0
print(f'RMSPE for LGBM on Test:{rmspe(lgbm_test_preds,test_prep_Y)}')

### Plot predictions

In [ ]:
fig ,axes =plt.subplots(2,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes[0].plot(train_data["Sales"], label='Train',color='blue')
axes[0].plot(test_data["Sales"], label='Test',color='orange')

axes[0].plot(lgbm_test_preds,label="LGBM",color='red')
axes[0].legend(loc='best')

axes[1].plot(test_data["Sales"], label='Test',color='orange')
axes[1].plot(lgbm_test_preds,label="LGBM",color='red')
axes[1].legend(loc='best')
plt.show()

### Fine Tuning

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'learning_rate': [0.01, 0.1],  # Learning rate for boosting
    'n_estimators': [100, 200, 300],  # Number of boosting iterations
    'reg_alpha': [0.0, 0.1, 0.5],  # L1 regularization term on weights
    'reg_lambda': [0.0, 0.1, 0.5],  # L2 regularization term on weights
    'max_depth': [-1, 5, 10],  # Maximum depth of a tree
}

# Generate all possible combinations of hyperparameters
param_combinations = list(product(*param_grid.values()))

# Create lists to store the results
best_params = None
best_model = None
best_rmspe = float('inf')

# Iterate over each parameter combination
for params in tqdm(param_combinations):
    # Create the LGBMRegressor model with the current hyperparameters
    lgbm_model = LGBMRegressor(random_state=42, **dict(zip(param_grid.keys(), params)))
    
    # Fit the model to the training data
    lgbm_model.fit(train_prep_X, train_prep_Y)
    
    # Make predictions on the test data
    lgbm_test_preds = lgbm_model.predict(test_prep_X)
    lgbm_test_preds = pd.Series(lgbm_test_preds, index=test_prep_X.index)
    lgbm_test_preds = lgbm_test_preds.clip(lower=0)
    lgbm_test_preds[test_prep_X["Open"] == 0] = 0
    # Calculate RMSPE
    rmspe_value = rmspe(lgbm_test_preds, test_prep_Y)
    
    # Check if the current model has the best RMSPE
    if rmspe_value < best_rmspe:
        best_rmspe = rmspe_value
        print(f"  Found new best RMSPE: {best_rmspe}")
        best_params = params
        best_model = lgbm_model

# Print the best hyperparameters and RMSPE
print(f"Best Parameters: {dict(zip(param_grid.keys(), best_params))}")
print(f"RMSPE for Best LGBM on Test: {best_rmspe}")

### Performance of best model

In [ ]:
lgbm_test_preds = best_model.predict(test_prep_X)
lgbm_test_preds = pd.Series(lgbm_test_preds, index= test_prep_X.index)
lgbm_test_preds = lgbm_test_preds.clip(lower=0)
lgbm_test_preds[test_prep_X["Open"] == 0] = 0
print(f'RMSPE for LGBM on Test:{rmspe(lgbm_test_preds,test_prep_Y)}')

### Plot predictions of best model

In [ ]:
fig ,axes =plt.subplots(2,1)
fig.set_figheight(6)
fig.set_figwidth(16)

axes[0].plot(train_data["Sales"], label='Train',color='blue')
axes[0].plot(test_data["Sales"], label='Test',color='orange')

axes[0].plot(lgbm_test_preds,label="LGBM",color='red')
axes[0].legend(loc='best')

axes[1].plot(test_data["Sales"], label='Test',color='orange')
axes[1].plot(lgbm_test_preds,label="LGBM",color='red')
axes[1].legend(loc='best')
plt.show()

---

# 8. <a name="8">**Choosing Best Model**</a>
(<a href="#0">Go to top</a>)

| Method                        | RMSPE                    | Model Type      |
|-------------------------------|-------------------------|------------------|
| **LGBM**                      | **0.1039**              | **ML**           |
| Prophet                       | 0.1366                  | Time Series      |
| SARIMAX                       | 0.1947                  | Time Series      |
| Auto Arima                    | 0.1999                  | Time Series      |
| average method                | 0.2700                  | Simple Model     |
| Arima                         | 0.2707                  | Time Series      |
| Seasonal Naive method         | 0.2785                  | Simple Model     |
| Naive method                  | 0.3849                  | Simple Model     |
| Drift method                  | 0.4765                  | Simple Model     |


## Summary

**The LGBM model outperformed other models in the competition, indicating that the provided features have significant predictive power for sales forecasting. The time series models, including Prophet, SARIMAX and Auto ARIMA, performed reasonably well but were less accurate than the LGBM model. The simple models, such as the average method and the Seasonal Naive method, provided baseline performance, while the Naive method, and Drift method showed relatively higher errors, suggesting limitations in capturing the complexities of the sales data.**

## <font color='green'>**Best model: LGBM Regressor**</font>

**The best-performing model in terms of RMSPE (Root Mean Square Percentage Error) is the LightGBM (LGBM) model, with an RMSPE of 0.1039. LGBM is a gradient boosting framework that uses decision trees as base learners. Its strong performance suggests that the features provided in the dataset have significant predictive power, and the LGBM model effectively captures the relationships between these features and the sales data.**

## <font color='green'>**Second Best model: Prophet**</font>

**The second-best model is Prophet, a time series forecasting model developed by Facebook, with an RMSPE of 0.1366. Prophet is known for its ability to handle various time series patterns such as seasonality, trends, and holidays. Its slightly higher error compared to LGBM may be due to the nature of the data, where the predictive features utilized by LGBM might be better suited for capturing the sales patterns.**

---